In [1]:
import pyspark
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from settings import *

# Training Model

In [2]:
# Sorting data by time
rdd =  sc.textFile(RATINGS_10M).map(lambda line: [float(x) for x in line.split('::')]).sortBy(lambda x: x[3],False)

../../data/HW2/ml-10M100K/ratings.dat


In [15]:
# Processing data to structure: Rating(user=62510, product=34148, rating=3.0)
train = rdd.zipWithIndex().filter(lambda x: x[-1] < size*0.6).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))

In [16]:
def computeUserBias(data):
    

Rating(user=62510, product=34148, rating=3.0)


In [17]:
# Training ALS Model
rank = 10
numIterations = 10
model = ALS.train(train, rank, numIterations)

# Model Evaluation

In [41]:
# Trying model evaluation

testdata = rdd.zipWithIndex().filter(lambda x: x[-1] > size*0.6).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))
test = testdata.map(lambda p: (p[0], p[1]))

In [42]:
predictions = model.predictAll(test).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = testdata.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)

((54040, 912), 4.367416929660967)
((31630, 1412), (4.0, 4.163697460854653))


In [43]:
MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 1.00137982702


# Recommendation System

In [ ]:
# Trying recommendation System 

#testdata = train.map(lambda p: (p[0], p[1]))
testUsers = train.map(lambda p: p[0])

In [ ]:
model.recommendProducts(testUsers, 100)

#predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
#ratesAndPreds = train.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)


In [ ]:
#MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
#print("Mean Squared Error = " + str(MSE))